In [1]:
import pandas as pd
import pickle
import os

In [2]:
from utils import haversine
from utils import join_data, train_test,hervy_dist, minutes_convert, haversine,taxi_demand_cluster, predict_lat_long, plt_map

In [3]:
dst_folder = os.path.join(os.getcwd(),"Processed_Data")

In [4]:
df = join_data(dst_folder)

In [5]:
df_model = pd.read_csv("Classical_ML_Models_test.csv")

In [6]:
df_model.drop("Unnamed: 0", axis=1, inplace=True)

In [7]:
df_model.sort_values(by= "Score", inplace=True)

In [8]:
df_model

,Model,Score,File_name
4,XGB,1890.300460,finalized_model_XGB.sav
3,Random_Forest,1897.904023,finalized_model_random_forest.sav
0,Linear_Regression,2240.509841,finalized_linear.sav
2,Decision_tree,2336.103194,finalized_model_decision_tree.sav
5,XGBF,2355.207638,finalized_model_random_forest.sav
1,KNN,2630.195285,finalized_model_knn.sav


In [9]:
filename = df_model.iloc[0]["File_name"]

In [10]:
df_test  = pd.read_csv("Test_Data.csv")
test_data = df_test.copy()

In [11]:
model = pickle.load(open(filename, 'rb'))
scaler = pickle.load(open('scaler.pkl','rb'))

In [16]:
def closest_prediction(df,df_test,model,index, scaler):
    df_test.drop("Unnamed: 0",axis=1, inplace=True)
    test_data = df_test.copy()
    df_test["Latitude"] = df_test["Latitude"]/90
    df_test["Longitude"] = df_test["Longitude"]/180
    df_test[['Hour', 'Day', 'Minutes_Identifier']] = scaler.fit_transform(df_test[['Hour', 'Day', 'Minutes_Identifier']])
    test_lat = df_test.iloc[[index]]
    pred = model.predict(test_lat)
    distance_to_prediction = haversine(list(pred[0]),list(test_data[["Latitude","Longitude"]].iloc[[index]].values[0]))
    df_cluster, df_centroids = taxi_demand_cluster(df,test_data.iloc[index]["Hour"], test_data.iloc[index]["Minutes_Identifier"],test_data.iloc[index]["Day"], 300, 20)
    dfc, df_label = predict_lat_long(list(test_data[["Latitude","Longitude"]].iloc[[0]].values[0]), df_centroids)
    if dfc.iloc[0]["Distance"] <=distance_to_prediction:
        prediciton = (dfc[["Latitude_1","Longitude_1"]].iloc[[0]])
    else:
        prediciton = (list(pred[0]))
    return(prediciton, dfc, distance_to_prediction)

In [17]:
closest_prediction(df.copy(),df_test.copy(), model, 2, scaler)

(   Latitude_1  Longitude_1
 1   37.787651  -122.411982,
    Latitude_1  Longitude_1     Distance
 1   37.787651  -122.411982    11.247471
 3   37.798034  -122.407090  1231.461711
 2   37.799196  -122.436135  2491.270064
 0   37.757384  -122.420285  3442.711457,
 3169.1050889110093)